## Загрузка данных

Импортируем нужные библиотеки или которые могут нам понадобиться.

In [ ]:
import pandas as pd
import numpy as np
import pylab as pl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Мы вывели таблицу, вывели значния методом describe() и инфо. Получили, что все таблицы распределены нормально: у нас нет пола с цифрой 2, нет лиц моложе 18 и мы видим, что лиц старше 65 нет, заплаты нет нулевой. В инфо видим, что все столбцы имеют правильный тип данных -> числовой. Можно перобразовать столбец "Возраст" в челочисленное для наглядности.

In [ ]:
df['Возраст'] = df['Возраст'].astype('int')

Посмотрим пропуски и дубликаты.

In [ ]:
df.isna().mean()

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

In [ ]:
df.duplicated().sum()

153

Вы видим, что у нас есть 153 явных дупликатов. Удалим их и восстановим порядок индексов.

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

Отделим признаки и целевой.

In [ ]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)

## Умножение матриц

Задание: Если признаки умножить на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки (наша матрица)

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Если матрицу умножить на обратную матрицу самой себе получим единичную матрицу. Порядок умножения не важен.

$$
XX^{-1}=X^{-1}X=E
$$

Если единичную матрицу умножить на любую матрицу получится таже матрица. Порядок умножения не важен.

$$
EX=XE=X
$$

**Ответ:** Не изменится. Ниже я привожу математическое решение и соотвестственно обоснование.  

**Обоснование:** 

У нас есть, что $$a = Xw$$ 
Тогда $$a_1 = XPw_1$$ так как чтобы получить преобразованную матрицу, мы матрицу X умножаем на обратимую P.

Мы знаем, по формуле выше что $$w = (X^TX)^{-1}X^Ty$$ Теперь найдем w_1

$$XPw_1=y$$ Нам нужно избавиться XP, чтобы получить w_1. 
Для этого мы для начала превращаем нащу матрицу в квадратную домнажая ее на транспонированную самой себе,

$$(XP)^TXPw_1=(XP)^Ty$$ 

а затем умножаем нашу квадратную матрицу на обратимую, чтобы получить E или 1. Формула выше.

$$(XP^TXP)^{-1}(XP^TXP)w_1=(XP^TXP)^{-1}(XP)^Ty$$

Получаем:

$$w_1=(XP^TXP)^{-1}(XP)^Ty$$

Теперь раскроем скобки наших параметров весов и скоратим.

$$w=(X^TX)^{-1}X^Ty=X^{-1}(X^T)^{-1}X^Ty=X^{-1}y$$
$$w_1=((XP)^TXP)^{-1}(XP)^Ty=(XP)^{-1}((XP)^T)^{-1}(XP)^Ty=(XP)^{-1}y=P^{-1}X^{-1}y=P^{-1}w$$

Подставим все значения в предсказания:

$$a=Xw$$
$$a_1=XPw_1=XPP^{-1}w=Xw$$

Видим, что у а_1 получился тот же ответ. Из этого следует, что:

$$a=a_1$$

## Алгоритм преобразования

**Алгоритм**

Создадим из признаков вектор X.  
Создадим квадратную матрицу из рандомных чисел P.  
Сделаем проверку.
Умножим X и P.  
Посмотрим метрику и напишем предположение.

In [ ]:
def checking_the_conversion(features):
    value = []
    X_features = features.values
    while True:
        P = np.random.randint(0, 10, size = (X_features.shape[1], X_features.shape[1]))
        P_reverse = np.linalg.inv(P)
        if np.allclose(np.dot(P, P_reverse), np.eye(P.shape[0])) is True:
            value.append('Массив обратимый')
            return X_features, P, value
            break
        value.append('Ошибка')

Проверим статус нашей матрицы. "value" нам покажет, если у нас вдруг наша матрица окажется не обратимой то в нее будет записана ошибка, а сама матрица пересоздается.

In [ ]:
X_features, P, value = checking_the_conversion(features)
print('Статус массива:', value)

Статус массива: ['Массив обратимый']


In [ ]:
XP_features = X_features @ P

Делим на обучающую и тестовую.

In [ ]:
XP_features_train, XP_features_valid, target_train, target_valid = train_test_split(
                                            XP_features, target, test_size=0.3, random_state=12345)
XP_features_train.shape[0], XP_features_valid.shape[0]

(3392, 1455)

In [ ]:
model = LinearRegression()
model.fit(XP_features_train, target_train)
XP_predictions = model.predict(XP_features_valid)
XP_r2 = r2_score(target_valid, XP_predictions)
XP_r2

0.43287552621820424

In [ ]:
XP_mse = mean_squared_error(target_valid, XP_predictions)
XP_mse

0.12972611526820765

На преобразованных данных у меня получилась метрика r2-0.43, MSE-0.13.

**Обоснование**

Мне кажется качество линейной регрессии не поменяется из-за того, что перемножая матрицы у нас меняется лишь их значения, а сами координаты остаются в тех же позициях, изменяясь лишь в цифрах. Это как пороги на ROC_AUC. Если мы домножим признаки на число, то распределение порогов у нас не изменится так же как и классы. Тут такой же принцип.

## Проверка алгоритма

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
                                                 features, target, test_size=0.3, random_state=12345)
features_train.shape[0], features_valid.shape[0]

(3392, 1455)

In [ ]:
numeric = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [ ]:
model.fit(features_train, target_train)
X_predictions = model.predict(features_valid)
X_r2 = r2_score(target_valid, X_predictions)
X_r2

0.43287552621918113

In [ ]:
X_mse = mean_squared_error(target_valid, X_predictions)
X_mse

0.1297261152679842

На обычных данных у меня получилась метрика r2-0.43, MSE-0.13.

In [ ]:
print()
print('Сравнение качества метрик на обычной выборке и после преобразования')
pd.DataFrame({'X_values': [X_r2, X_mse], 'XP_values': [XP_r2, XP_mse]},index=['r2', 'mse'])


Сравнение качества метрик на обычной выборке и после преобразования


,X_values,XP_values
r2,0.432876,0.432876
mse,0.129726,0.129726


Как мы видим, метрика не изменилась!

# Общий вывод:

1. Мы Прочитали файл, сделали небольшую предобработку и отделили признаки и целевой признак.
2. Прежде чем начинать делать практическую часть, мы решили уравнение математически и ответом оказалось, то что при преобразовании матрицы предсказания у нас не изменятся.
3. Мы создали преобразованную матрицу путем умножения нашей исходной матрицы на рандомную квадратную обратимую матрицу. На ней обучили выборку и получили метрику качества.
4. Обучили алгоритм на нашей исходной матрице и получили на ней метрику качетсва.
5. Получили одинаковые результаты, что на исходной, что на преобразованной матрице. Метрика осталась неизменной, что нам доказало решеннное уравнение.